# **Importing modules**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#confirming gpu connection
import tensorflow as tf
device_name=tf.test.gpu_device_name()
print(device_name)

In [ ]:
!pip install keras==2.3.1
!pip install tensorflow==2.1.0
!pip install keras_applications==1.0.8
!pip install image-classifiers==1.0.0
!pip install segmentation-models
!pip install efficientnet==1.0.0
!pip install segmentation_models
!pip install tta_wrapper

     |████████████████████████████████| 377 kB 5.3 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires keras<2.8,>=2.7.0rc0, but you have keras 2.3.1 which is incompatible.
     |████████████████████████████████| 421.8 MB 25 kB/s 
     |████████████████████████████████| 448 kB 69.8 MB/s 
     |████████████████████████████████| 3.8 MB 41.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2c42140195cc8dfbe78ac39adfb48b997e85a6274a51794e00269754803dcbfc
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting unins

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns = None
%matplotlib inline
mlp.rcParams["figure.figsize"] = (25, 5)
plt.style.use('seaborn')
import sys 
import os
import random
import shutil
import zipfile
import tensorflow
from glob import glob
from datetime import datetime
from tqdm import tqdm_notebook

seed = 123
random.seed(seed)
np.random.seed(seed)
import cv2
from google.colab.patches import cv2_imshow

from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

from tensorflow.python.keras import backend as K 

from tensorflow import keras
from keras import regularizers, optimizers, Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Dense, GlobalAveragePooling2D
from keras.optimizers import *

from keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from tta_wrapper import tta_segmentation
from tensorflow.keras.utils import Sequence


# **Data Pre Processing**

In [ ]:
#to extract from zip files use this


#DRIVE_ZIP_DIR = "/content/drive/My Drive/Datasets/caliche"
# source_file_path = DRIVE_ZIP_DIR+ '/severstal-steel-defect-detection.zip'
# print("Extracting contents of zip file.")
# zip_ref = zipfile.ZipFile(source_file_path, 'r')
# zip_ref.extractall(DRIVE_ZIP_DIR)
# zip_ref.close()

# print("Done..")

In [ ]:
PATH_TO_DATA="/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection"
PATH_TO_PROJECT='/content/drive/My Drive/metal-sheet-defect'

In [ ]:
file_path = PATH_TO_DATA+'/train.csv'
raw_data = pd.read_csv(file_path)

# **Data Cleaning**

Handle missing values, outliers, rare values and drop the unnecessary features that do not carry useful information.

In [ ]:
print("The number of records are : ", raw_data.shape[0])
print("The number of features are : ", raw_data.shape[1])
print("The list of features is : ", raw_data.columns)
raw_data.head()

In [ ]:
def RLE_To_ImageMask(mask_rle, shape=(1600, 256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width, height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    # Splitting the run-length encoding
    s = mask_rle.split()
    # Creating a np array for start pixel and its length
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    # Reducing the start by 1
    starts -= 1
    # Creating a np array for end pixel
    ends = starts + lengths
    # Creating a img mask
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    # Entering 1 at the place for the defect pixels
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1

    return img.reshape(shape).T     


In [ ]:
def Add_Undefected_Images(data): 
    """ 
        Takes the dataframe as input and return the updated dataframe which contain both defected and undefected images information 
    """ 
    # Count the image file names for the given directory
    count = 0
    folder = "/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images"
    filename_list = []
    for filename in os.listdir(folder):
        if filename not in data["ImageId"].values:
            filename_list.append(filename)
        count += 1
    # Print the total number of images present in the directory
    print("No. of Training images provided : ", count)
    print("No. of Undefected images : ", len(filename_list))
    
    # Creating the dictionary that contain the undefected images details
    dictionary = {}
    dictionary.update({"ImageId": filename_list, "ClassId": 0, "EncodedPixels": 0, "mask_pixel_sum": 0})
    data_undefected = pd.DataFrame(dictionary)

    # Concatinate the the defected and undefected images in the single dataframe called data.
    data = pd.concat([data, data_undefected])
    
    return data

In [ ]:
# Calculating the sum of defected pixels for the defected image
raw_data["mask_pixel_sum"] = raw_data.apply(lambda x: RLE_To_ImageMask(x["EncodedPixels"]).sum(), axis = 1)
# Adding details for the undefected images
raw_data = Add_Undefected_Images(raw_data)

In [ ]:
raw_data["binary_class"] = raw_data["ClassId"].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
# Checking the features and no. of records in the dataset after including the details for missing images

print("The number of records are : ", raw_data.shape[0])
print("The number of features are : ", raw_data.shape[1])
print("The list of features is : ", raw_data.columns)

raw_data.tail()

In [ ]:
print("No. of images with multiple class of defects is : ", raw_data.shape[0] - 12568)
print("The basic info about the raw data is : \n")
raw_data.info()


In [ ]:
# Checking the missing values in the dataset
print("No. of missing values in the dataset : \n", raw_data.isnull().sum())

# **Exploratory Data Analysis**

In [ ]:
# Plotting a countplot to visualize the distribution of reviews based on Rating Score
df_binary_clf = raw_data[["ImageId", "binary_class"]].drop_duplicates()
sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(25, 5))
sns.countplot(x="binary_class", data=df_binary_clf, ax=axes)
plt.title("DEFECT vs NO DEFECT", fontsize='xx-large')
plt.xlabel("CLASSES")
plt.ylabel("BINARY CLASS COUNT")
plt.show() 

# **Analysing the training data-Types of defects & distribution**

In [ ]:
sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(25, 5))

sns.countplot(x="ClassId", data=raw_data, ax=axes)

plt.title("DISTRIBUTION OF DEFECT CLASS IN THE TRAINING DATA", fontsize='xx-large')
plt.xlabel("CLASSES")
plt.ylabel("DEFFECT CLASS COUNT")
plt.show() 

# **Scatter Plat for Mask Pixel Sum**

Defect1

In [ ]:
a = [i for i in raw_data[raw_data["ClassId"] == 1]["mask_pixel_sum"].values]
a = sorted(a)

sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(25, 5))

plt.title("Defect Class 1 Mask Sum Threshold Graph")
plt.scatter(a, range(len(a)))

Defect 2


In [ ]:
a = [i for i in raw_data[raw_data["ClassId"] == 2]["mask_pixel_sum"].values]
a = sorted(a)

sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(25, 5))

plt.title("Defect Class 1 Mask Sum Threshold Graph")
plt.scatter(a, range(len(a)))

Defect3

In [ ]:
a = [i for i in raw_data[raw_data["ClassId"] == 3]["mask_pixel_sum"].values]
a = sorted(a)

sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(25, 5))

plt.title("Defect Class 1 Mask Sum Threshold Graph")
plt.scatter(a, range(len(a)))

Defect4

In [ ]:
a = [i for i in raw_data[raw_data["ClassId"] == 4]["mask_pixel_sum"].values]
a = sorted(a)

sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(25, 5))

plt.title("Defect Class 1 Mask Sum Threshold Graph")
plt.scatter(a, range(len(a)))

In [ ]:
def Show_Images(dataset, defect_type, start, end):
    """
        Showing the images of different defect type.
    """

    print(f"Defect Class {defect_type}")
    image_1 = dataset[dataset["ClassId"] == defect_type][start:end]["ImageId"].values
    for i in image_1:
        print("Image id : ", i)
        sns.set(style="darkgrid")
        plt.figure(figsize=(20, 4))
        # Loading the images one by one from the directory.
        image_read = cv2.imread("/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images/" + i)
        image_read = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)
        # Show the graph
        plt.imshow(image_read) 


class 0

In [ ]:
Show_Images(raw_data, 0, 20, 24)

class 1

In [ ]:
Show_Images(raw_data, 1, 20, 24)

class 2

In [ ]:
Show_Images(raw_data, 2, 20, 24)

class 3

In [ ]:
Show_Images(raw_data, 3, 20, 24)

class 4

In [ ]:
Show_Images(raw_data, 4, 7, 11)

# **View Masking**

In [ ]:
#for def1
Encoded = raw_data[raw_data["ClassId"] == 1]["EncodedPixels"].values[35]
print("Masked Image Defect Class 1")

Masked_image = RLE_To_ImageMask(Encoded)
plt.figure(figsize=(20, 4))
plt.title("Sample Image Mask for Defect Class 1", fontsize='x-large')
plt.imshow(Masked_image, cmap="viridis")

In [ ]:
#for def2
Encoded = raw_data[raw_data["ClassId"] == 2]["EncodedPixels"].values[35]
print("Masked Image Defect Class 1")

Masked_image = RLE_To_ImageMask(Encoded)
plt.figure(figsize=(20, 4))
plt.title("Sample Image Mask for Defect Class 1", fontsize='x-large')
plt.imshow(Masked_image, cmap="viridis")

In [ ]:
#for def3
Encoded = raw_data[raw_data["ClassId"] == 3]["EncodedPixels"].values[35]
print("Masked Image Defect Class 1")

Masked_image = RLE_To_ImageMask(Encoded)
plt.figure(figsize=(20, 4))
plt.title("Sample Image Mask for Defect Class 1", fontsize='x-large')
plt.imshow(Masked_image, cmap="viridis")

In [ ]:
#for def4
Encoded = raw_data[raw_data["ClassId"] == 4]["EncodedPixels"].values[35]
print("Masked Image Defect Class 1")

Masked_image = RLE_To_ImageMask(Encoded)
plt.figure(figsize=(20, 4))
plt.title("Sample Image Mask for Defect Class 1", fontsize='x-large')
plt.imshow(Masked_image, cmap="viridis")

# **Feature engineering to prepare the data for algorithms**

In [ ]:
# Creating a new feature by combining the image name and class
raw_data["ImageId_ClassId"] = raw_data["ImageId"] + "_" + raw_data["ClassId"].astype(str) 
# Sorting the df based on image name and class
raw_data = raw_data.sort_values(by=["ImageId_ClassId"], axis=0, ignore_index=True)
raw_data.head()

In [ ]:
# Grouping the ImageId together to extract IDs with multiple defect classes
img_class_data = raw_data.groupby(["ImageId"])["ClassId"].agg(["unique"]).reset_index()

# Renaming the feature to "Class"
img_class_data.rename(columns = {"unique": "Class"}, inplace = True)

# Adding feature for each class
img_class_data["Class_0"] = 0
img_class_data["Class_1"] = 0
img_class_data["Class_2"] = 0
img_class_data["Class_3"] = 0
img_class_data["Class_4"] = 0

# Boolean feature indicating if defect exists
img_class_data["Any_Class"] = 0

In [ ]:
# Entering 1 in the respective Class feature and binary "Any_Class" feature

for idx, row in img_class_data.iterrows():
    if 0 in row.Class:
        img_class_data.at[idx, 'Class_0'] = 1
    if 1 in row.Class:
        img_class_data.at[idx, 'Class_1'] = 1
        img_class_data.at[idx, 'Any_Class'] = 1
    if 2 in row.Class:
        img_class_data.at[idx, 'Class_2'] = 1
        img_class_data.at[idx, 'Any_Class'] = 1
    if 3 in row.Class:
        img_class_data.at[idx, 'Class_3'] = 1
        img_class_data.at[idx, 'Any_Class'] = 1
    if 4 in row.Class:
        img_class_data.at[idx, 'Class_4'] = 1
        img_class_data.at[idx, 'Any_Class'] = 1

img_class_data["Sum_of_Defects"] = img_class_data["Class_1"] + img_class_data["Class_2"] + img_class_data["Class_3"] + img_class_data["Class_4"]
img_class_data.head()

In [ ]:
# basic informations regarding classes

print("no. of unique Images is : ", len(img_class_data))

print("no. of Images with defects is : ", len(img_class_data[img_class_data["Any_Class"] == 1]))
print("no. of Images without defects is : ", len(img_class_data[img_class_data["Any_Class"] == 0]))

print("no. of Images with Class Defect 1 is : ", len(img_class_data[(img_class_data["Class_1"] == 1) & (img_class_data["Sum_of_Defects"] == 1)]))
print("no. of Images with Class Defect 2 is : ", len(img_class_data[(img_class_data["Class_2"] == 1) & (img_class_data["Sum_of_Defects"] == 1)]))
print("no. of Images with Class Defect 3 is : ", len(img_class_data[(img_class_data["Class_3"] == 1) & (img_class_data["Sum_of_Defects"] == 1)]))
print("no. of Images with Class Defect 4 is : ", len(img_class_data[(img_class_data["Class_4"] == 1) & (img_class_data["Sum_of_Defects"] == 1)]))

print("no. of Images with 1 Class Defects is : ", len(img_class_data[img_class_data["Sum_of_Defects"] == 1]))
print("no. of Images with 2 Class Defects is : ", len(img_class_data[img_class_data["Sum_of_Defects"] == 2]))
print("no. of Images with 3 Class Defects is : ", len(img_class_data[img_class_data["Sum_of_Defects"] == 3]))
print("no. of Images with 4 Class Defects is : ", len(img_class_data[img_class_data["Sum_of_Defects"] == 4]))

# **Training the model**

**1. Binary Classification**

**Using a smaller model**

In [ ]:
columns = ["ImageId", "Class_1", "Class_2", "Class_3", "Class_4", "Any_Class", "Sum_of_Defects"]
y_columns = ["Any_Class"]
df = img_class_data[columns]
#id in the form of npy
df['ImageId']=df['ImageId'].map(lambda element: element[:-4]+'.npy')
df

In [ ]:
df_small=df.iloc[:5000]
df_small

In [ ]:
# Plotting a countplot to visualize the distribution of reviews based on Rating Score 
#to ensure even distributtion of defected and non-defected images in the smaller dataset
sns.set(style="darkgrid")
fig, axes = plt.subplots(1, 1, figsize=(25, 5))
sns.countplot(x="Any_Class", data=df_small, ax=axes)
plt.title("DEFECT vs NO DEFECT", fontsize='xx-large')
plt.xlabel("CLASSES")
plt.ylabel("BINARY CLASS COUNT")
plt.show() 

In [ ]:
train_df, val_df = train_test_split(df_small, stratify=df_small[["Any_Class"]], test_size=0.20, random_state=42)
print("Train Data Shape :", train_df.shape, "Val Data Shape :", val_df.shape)

In [ ]:
#for the whole dataset use this 

# train_df, val_df = train_test_split(df, stratify=df[["Any_Class"]], test_size=0.20, random_state=42)
# print("Train Data Shape :", train_df.shape, "Val Data Shape :", val_df.shape)
# #NOTATIONS
# partition={'train':train_df['ImageId'].tolist(),'validation':val_df['ImageId'].tolist()}
# labels=dict(zip(df.ImageId,df.Any_Class))

In [ ]:
partition_small={'train':train_df['ImageId'].tolist(),'validation':val_df['ImageId'].tolist()}
labels_small=dict(zip(df_small.ImageId,df_small.Any_Class))

**Designing and fitting the model using smaller dataset**

In [ ]:
#Data generator
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=16, dim=(256,1600), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            size=len(ID)
            X[i,] = np.load('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images_np/' + ID)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
#MODEL 4
from tensorflow import keras
from keras.models import Sequential
# Parameters
params = {'dim': (256,1600),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition_small['train'], labels_small, **params)
validation_generator = DataGenerator(partition_small['validation'], labels_small, **params)


# Design model
model1 = tensorflow.keras.Sequential()
model1.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 1600, 3)))
model1.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model1.add(keras.layers.MaxPooling2D((2, 2)))
model1.add(keras.layers.Flatten())
model1.add(keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
model1.add(keras.layers.Dense(2, activation='softmax'))
	# compile model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train model on dataset
history=model1.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    epochs=10,
                   )

In [ ]:
import matplotlib
import matplotlib.pyplot
def summarize_diagnostics(history):
	# plot loss
	matplotlib.pyplot.subplot(211)
	matplotlib.pyplot.title('Cross Entropy Loss')
	matplotlib.pyplot.plot(history.history['loss'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	matplotlib.pyplot.subplot(212)
	matplotlib.pyplot.title('Classification Accuracy')
	matplotlib.pyplot.plot(history.history['accuracy'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file

	matplotlib.pyplot.savefig('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/' + 'adam_4_10epochs.png')
	matplotlib.pyplot.close()
 
summarize_diagnostics(history)
model1.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_4_small_data.h5')

In [ ]:
from keras.models import load_model
new_model = model1
history=new_model.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    epochs=30,
                  initial_epoch=10)
new_model.save("updated_model_4_small_data.h5")

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 62 steps
Epoch 11/30
250/250 [==============================] - 432s 2s/step - loss: 0.0047 - accuracy: 0.9998 - val_loss: 1.9416 - val_accuracy: 0.7581
Epoch 12/30
250/250 [==============================] - 228s 911ms/step - loss: 0.0055 - accuracy: 0.9995 - val_loss: 1.9869 - val_accuracy: 0.7480
Epoch 13/30
250/250 [==============================] - 228s 913ms/step - loss: 0.0029 - accuracy: 0.9998 - val_loss: 2.0334 - val_accuracy: 0.7470
Epoch 14/30
250/250 [==============================] - 228s 913ms/step - loss: 0.0061 - accuracy: 0.9995 - val_loss: 1.9857 - val_accuracy: 0.7440
Epoch 15/30
250/250 [==============================] - 228s 912ms/step - loss: 0.0047 - accuracy: 0.9995 - val_loss: 2.0258 - val_accuracy: 0.7440
Epoch 16/30
250/250 [==============================] - 228s 911ms/step - loss: 0.0023 - accuracy: 0.9998 - val_loss: 2.2835 - val_accuracy: 0.7490
Epoch 17/30
250/250 [=======

In [ ]:
def summarize_diagnostics(history):
	# plot loss
	matplotlib.pyplot.subplot(211)
	matplotlib.pyplot.title('Cross Entropy Loss')
	matplotlib.pyplot.plot(history.history['loss'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	matplotlib.pyplot.subplot(212)
	matplotlib.pyplot.title('Classification Accuracy')
	matplotlib.pyplot.plot(history.history['accuracy'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file

	matplotlib.pyplot.savefig('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/' + 'updated_adam_4_30epochs.png')
	matplotlib.pyplot.close()
 
summarize_diagnostics(history)

In [ ]:
new_model.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/updated_model_4_small_data.h5')

In [ ]:
#SMALL MODEL 2
#lr 0.001
from tensorflow import keras
from keras.models import Sequential
import matplotlib.pyplot

# Parameters
params = {'dim': (256,1600),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition_small['train'], labels_small, **params)
validation_generator = DataGenerator(partition_small['validation'], labels_small, **params)

# Design model
model2 = tensorflow.keras.Sequential()
model2.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 1600, 3)))
model2.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(keras.layers.MaxPooling2D((2, 2)))
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
model2.add(keras.layers.Dense(2, activation='softmax'))
	# compile model
opt = keras.optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])



# Train model on dataset
history=model2.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    epochs=3,
                   )
def summarize_diagnostics(history):
	# plot loss
	matplotlib.pyplot.subplot(211)
	matplotlib.pyplot.title('Cross Entropy Loss')
	matplotlib.pyplot.plot(history.history['loss'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	matplotlib.pyplot.subplot(212)
	matplotlib.pyplot.title('Classification Accuracy')
	matplotlib.pyplot.plot(history.history['accuracy'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	matplotlib.pyplot.savefig(filename + '_plot.png')
	matplotlib.pyplot.close()
 
summarize_diagnostics(history)
model2.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_2_small_data.h5')

Epoch 1/3
250/250 [==============================] - 4149s 17s/step - loss: 1966.6694 - accuracy: 0.5203 - val_loss: 0.6925 - val_accuracy: 0.5202
Epoch 2/3
250/250 [==============================] - 272s 1s/step - loss: 0.6926 - accuracy: 0.5200 - val_loss: 0.6924 - val_accuracy: 0.5202
Epoch 3/3
250/250 [==============================] - 265s 1s/step - loss: 0.6925 - accuracy: 0.5200 - val_loss: 0.6924 - val_accuracy: 0.5192


In [ ]:
def summarize_diagnostics(history):
	# plot loss
	matplotlib.pyplot.subplot(211)
	matplotlib.pyplot.title('Cross Entropy Loss')
	matplotlib.pyplot.plot(history.history['loss'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	matplotlib.pyplot.subplot(212)
	matplotlib.pyplot.title('Classification Accuracy')
	matplotlib.pyplot.plot(history.history['accuracy'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	matplotlib.pyplot.savefig(filename + '_plot.png')
	matplotlib.pyplot.close()
 
summarize_diagnostics(history)
model2.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_2_small_data.h5')

In [ ]:

class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=16, dim=(256,1600), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            size=len(ID)
            X[i,] = np.load('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images_np/' + ID)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
#SMALL MODEL 3
#lr 0.0005
from tensorflow import keras
from keras.models import Sequential
import matplotlib.pyplot

# Parameters
params = {'dim': (256,1600),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition_small['train'], labels_small, **params)
validation_generator = DataGenerator(partition_small['validation'], labels_small, **params)

# Design model
model3 = tensorflow.keras.Sequential()
model3.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 1600, 3)))
model3.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model3.add(keras.layers.MaxPooling2D((2, 2)))
model3.add(keras.layers.Flatten())
model3.add(keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
model3.add(keras.layers.Dense(2, activation='softmax'))
	# compile model
opt = keras.optimizers.Adam(learning_rate=0.0005)
model3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])



# Train model on dataset
history=model3.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    epochs=3,
                   )
def summarize_diagnostics(history):
	# plot loss
	matplotlib.pyplot.subplot(211)
	matplotlib.pyplot.title('Cross Entropy Loss')
	matplotlib.pyplot.plot(history.history['loss'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	matplotlib.pyplot.subplot(212)
	matplotlib.pyplot.title('Classification Accuracy')
	matplotlib.pyplot.plot(history.history['accuracy'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	matplotlib.pyplot.savefig(PATH_TO_DATA+'/adam_lr2.png')
	matplotlib.pyplot.close()
 
summarize_diagnostics(history)
model3.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_3_small_data.h5')

Epoch 1/3
250/250 [==============================] - 4789s 19s/step - loss: 818.4438 - accuracy: 0.5742 - val_loss: 1.5086 - val_accuracy: 0.5262
Epoch 2/3
250/250 [==============================] - 4648s 19s/step - loss: 1.2233 - accuracy: 0.5210 - val_loss: 1.0974 - val_accuracy: 0.5292
Epoch 3/3
250/250 [==============================] - 4643s 19s/step - loss: 0.6739 - accuracy: 0.5420 - val_loss: 0.8047 - val_accuracy: 0.5333


In [ ]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=16, dim=(256,1600), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            size=len(ID)
            X[i,] = np.load('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images_np/' + ID)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
#########
#MODEL 5
#VGG 3 layers
#Parameters
params = {'dim': (256,1600),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition_small['train'], labels_small, **params)
validation_generator = DataGenerator(partition_small['validation'], labels_small, **params)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 1600, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


history=model.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    epochs=5,
                   )
def summarize_diagnostics(history):
	# plot loss
	matplotlib.pyplot.subplot(211)
	matplotlib.pyplot.title('Cross Entropy Loss')
	matplotlib.pyplot.plot(history.history['loss'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	matplotlib.pyplot.subplot(212)
	matplotlib.pyplot.title('Classification Accuracy')
	matplotlib.pyplot.plot(history.history['accuracy'], color='blue', label='train')
	matplotlib.pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	matplotlib.pyplot.savefig(PATH_TO_DATA+'/adam_lr5.png')
	matplotlib.pyplot.close()
summarize_diagnostics(history)
model.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_5_small_data.h5')

**Convert jpg tp npy**

In [ ]:
import os
from numpy import save
dir=os.listdir('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images/')
dir.sort()
for item in dir:
  if os.path.isfile('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images/'+item):
    img=Image.open('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images/'+item).convert("RGB")
    img=np.array(img)
    size=len(item)
    np.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images_np/'+item[:size-4]+'.npy',img)
    del(img)



**Designing and fitting the model using entire dataset**

In [ ]:
#on the whole dataset
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=16, dim=(256,1600), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            size=len(ID)
            X[i,] = np.load('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/train_images_np/' + ID)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
#MODEL 0
from tensorflow import keras
from keras.models import Sequential
# Parameters
params = {'dim': (256,1600),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = tensorflow.keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 1600, 3)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(2, activation='softmax'))
	# compile model
opt = tensorflow.keras.optimizers.SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


# Train model on dataset
model.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 628 steps, validate for 157 steps
628/628 [==============================] - 4347s 7s/step - loss: nan - accuracy: 0.4688 - val_loss: nan - val_accuracy: 0.4697


In [ ]:
#MODEL 1
from tensorflow import keras
from keras.models import Sequential
# Parameters
params = {'dim': (256,1600),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model1 = tensorflow.keras.Sequential()
model1.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 1600, 3)))
model1.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model1.add(keras.layers.MaxPooling2D((2, 2)))
model1.add(keras.layers.Flatten())
model1.add(keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
model1.add(keras.layers.Dense(2, activation='softmax'))
	# compile model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train model on dataset
model1.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False
                   )

628/628 [==============================] - 3848s 6s/step - loss: 550.8049 - accuracy: 0.6961 - val_loss: 0.6496 - val_accuracy: 0.7536


In [ ]:
#MODEL 2
from tensorflow import keras
from keras.models import Sequential
# Parameters
params = {'dim': (256,1600),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model2 = tensorflow.keras.Sequential()
model2.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 1600, 3)))
model2.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(keras.layers.MaxPooling2D((2, 2)))
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
model2.add(keras.layers.Dense(2, activation='softmax'))
	# compile model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 

# Train model on dataset
history=model2.fit(training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    epochs=10,
                   )
summarize_diagnostics(history)

In [ ]:
model.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_0.h5')

In [ ]:
model1.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_1.h5')

In [ ]:
model2.save('/content/drive/My Drive/Datasets/caliche/severstal-steel-defect-detection/model_2.h5')